In [ ]:
from sensormap_client import ApiClient, Configuration, models
from helpers.wrapper import get_jwt_token
from sensormap_client.api.default_api import DefaultApi
import pandas as pd
import geopandas as gpd
import json

SERVER = "https://soil-sensormap.epfl.ch" 
# SERVER = "http://soil:88"

auth_token = get_jwt_token(SERVER)
config = Configuration(host=SERVER, access_token=auth_token)
api = DefaultApi(ApiClient(configuration=config))

# Test getting some projects, to see if we're connected
print("Connected") if api.get_all_projects() is not None else print("Not connected")

In [ ]:
# Print out the projects and areas
projects = api.get_all_projects()
for project in projects:
    print(f"Project: {project.name}")

areas = api.get_all_areas()
for area in areas:
    print(f"Area: {area.name} (Project: {area.project.name})")

In [ ]:
# # Create a project

# # We need to use the ProjectCreate model to create
# project = models.ProjectCreate(
#     name="Soil Amendment",
#     description="The soil amendement project aims at investigatingthe effect of different organic amendements on agricultural soils. This project is in collaboration with the Nestlé research center"
# )

# # Send it to the Database
# created_project = api.create_one_project(project)
# print("Created project. ID:", created_project.id)

In [ ]:
params = {"name": "Moudon"}
moudon = api.get_all_areas(filter=json.dumps(params))[0]
moudon.id

In [ ]:
help(models.PlotCreate)

In [ ]:
df = pd.read_csv("moudon.csv")

# Load coordinates, convert them from latlong (4326) to SwissGrid (2056)
coords = pd.read_csv("moudon-coords.csv")
coords = gpd.GeoDataFrame(coords, geometry=gpd.GeoSeries.from_xy(coords['longitude'], coords['latitude']), crs=4326).to_crs(2056)

# Join the tables on the plot column
combined = df.merge(coords[['plot', 'geometry']], left_on='Plot', right_on='plot', how="inner")

In [ ]:
df

In [ ]:
for i, row in combined.iterrows():
    # print(row)
    plot = models.PlotCreate(area_id=moudon.id, name=row.Plot, coord_x=row.geometry.x, coord_y=row.geometry.y, coord_z=0)
    api.create_one_plot(plot)